# N2O actinometry experiment | It-factor computation

In [4]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

## user interface
N2O_flow = np.array([804,1034,1250,1452,1641,1818,1985,2143,2292,2432]) # sccm
N2_flow = np.array([6696,6466,6250,6048,5859,5682,5515,5357,5208,5068]) # sccm
NO_conc = np.array([5.300,6.413,7.800,8.983,9.893,10.919,11.781,12.319,13.044,13.566]) # ppb

Temp = 25 # °C

flow_inlet = 7500  # inlet flow CI-API-TOF, sccm

R = 0.78  #0.78 cm, 3/4 inch inner radius


## parameters
T = Temp + 273.15  # K
K6 = 2.0 * 1e-11 * np.exp(130 / T)  # cm3 mol-1 s-1
K7 = 7.6 * 1e-11  # cm3 mol-1 s-1
K8 = 4.3 * 1e-11  # cm3 mol-1 s-1
K9 = 6.0 * 1e-12  # cm3 mol-1 s-1

phi_N2O = 1  # quantum yield
sigma_N2O = 1.43 * 1e-19  # absorption cross section, cm2

Total = N2_flow + N2O_flow # sccm
NOx = NO_conc  # ppb

mixing_N2 = N2_flow / (N2O_flow + N2_flow)
mixing_N2O = N2O_flow / (N2O_flow + N2_flow)

## It-product calculation
It_N2O = (K6 * mixing_N2 + (K7 + K8 + K9) * mixing_N2O) * NOx * 1e-9 / (2 * K7 * sigma_N2O * phi_N2O * mixing_N2O ** 2);
It = It_N2O * (N2_flow + N2O_flow) / flow_inlet

print('It_non_corr_mean', "{:.2E}".format(np.mean(It)))
print('It_non_corr_median', "{:.2E}".format(np.median(It)))
print('\nIt_non_corr', [f"{val:.2E}" for val in It])

It_non_corr_mean 5.23E+11
It_non_corr_median 4.75E+11

It_non_corr ['8.70E+11', '6.81E+11', '6.02E+11', '5.42E+11', '4.90E+11', '4.59E+11', '4.32E+11', '4.01E+11', '3.83E+11', '3.65E+11']


In [5]:
## geometry correction factor

N2O_conc = 96060. / 1.38e-23 / (Temp + 273.15) / 1e6 * mixing_N2O
m = len(mixing_N2O)

from scipy import integrate

fx = []
for i in range(m):
    f = lambda y, x: np.exp(-sigma_N2O * N2O_conc[i] * (np.sqrt(R ** 2 - y ** 2) + x))
    fx1 = integrate.dblquad(f, -R, R, lambda x: -np.sqrt(R ** 2 - x ** 2), lambda x: np.sqrt(R ** 2 - x ** 2))
    fx.append(fx1[0])

q = np.transpose(fx)

K = q / R ** 2 / np.pi

print("K values:", [f"{val:.2f}" for val in K])

K values: ['0.80', '0.75', '0.71', '0.67', '0.64', '0.62', '0.59', '0.57', '0.55', '0.54']


In [7]:
## corrected It-product calculation
It_corr = It / K

It_final_mean = np.mean(It_corr)
It_final_median = np.median(It_corr)
It_corr_low = min(It_corr)
It_corr_high = max(It_corr)

print('It_final_mean', "{:.2E}".format(It_final_mean))
print('It_final_median', "{:.2E}".format(It_final_median))
print('\nIt_corr', [f"{val:.2E}" for val in It_corr])
print('\nIt_non_corr', [f"{val:.2E}" for val in It])

It_final_mean 7.95E+11
It_final_median 7.52E+11

It_corr ['1.09E+12', '9.08E+11', '8.48E+11', '8.03E+11', '7.60E+11', '7.44E+11', '7.27E+11', '7.01E+11', '6.93E+11', '6.79E+11']

It_non_corr ['8.70E+11', '6.81E+11', '6.02E+11', '5.42E+11', '4.90E+11', '4.59E+11', '4.32E+11', '4.01E+11', '3.83E+11', '3.65E+11']
